# Requests et BeautifulSoup

## Installation Des Packages

In [1]:
# Si nous n'avez pas encore installé ces packages il faut enlever les les lignes suivantes en commentaire etles executer
# !pip install requests --upgrade --quiet
# !pip install beautifulsoup4 --upgrade --quiet
# !pip install pandas --quiet

## Importation des Packages

In [2]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

## Comment sa fonctionne

La première étape consiste à utiliser le package <code>**Requests**</code> pour télécharger le code HTML de notre page Web (à l'aide de la fonction <code>**requests.get**</code>) qui donne un objet de réponse. Nous pouvons voir si nous pouvons télécharger la page Web ou non en vérifiant la réponse. Statut de notre objet. (Il devrait être compris entre <code>**200 et 299**</code>). Ensuite, nous allons convertir cet objet de réponse en un objet <code>**BeautifulSoup**</code> en utilisant le constructeur <code>**BeautifulSoup()**</code>. Nous utiliserons cet objet pour inspecter notre document et extraire les données souhaitées. <code>**BeautifulSoup**</code> nécessite un argument supplémentaire appelé <code>**parser**</code>. (En bref, **BeautifulSoup** peut également être utilisé pour extraire des informations d'autres langages de balisage. Le **parser** par défaut est <code>**"html.parser"**</code>). Trouvez ci-dessous une fonction d'exemple indépendante qui effectuera la tâche ci-dessus: 
* de téléchargement des pages, 
* de vérification des réponses
d'analyse à l'aide de <code>**BeautifulSoup()**</code>.

In [6]:
def get_item_page(items_url):
    # télécharger la page
    response = requests.get(items_url)
    # vérifier le succès de réponse
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(items_url))
    # Parser la réponse à l'aide de beaufifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

## Scraper la liste des articles de notre site Web cible
Dans notre cas, nous allons scraper https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page} Nous obtiendrons une liste des produits en vol. Nous obtiendrons un **titre d'article principal**, un **prix**, **des notes** et une **URL** de page pour chaque article. Pour **chaque article**, nous obtiendrons plus de détails sur le produit à partir de la page de l'article (si nécessaire). Nous allons créer un dataframe de Pandas et un fichier **Json** avec les détails de l'élément. Pour chaque article, nous créerons également un fichier **CSV** au format indicatif suivant:

nom, prix, URL
Article JUNIO Fleur Préservée Avec Boîtes, 5.50 - 6.50, https://www.alibaba.com/product-detail/Item-JUNIO-Preserved-Flower-With-Boxes_62339028105.html?s=p

In [7]:
url = "https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab=%7Bpage%7D"

In [8]:
get_item_page(url)

 <!-- tangram:4926 begin-->
<!DOCTYPE html>

<html class="rwd">
<head>
<script>
                window.__setBBParams = function(keyName, timeVal){
            if( typeof(Performance) != 'undefined' ){
                window.__BB = window.__BB || { mode: -1 };
                window.__BB['extendParams'] = window.__BB['extendParams'] || {};
                timeVal = timeVal || new Date().getTime() - performance.timing.navigationStart;
                window.__BB['extendParams'][keyName]= timeVal;
            }
        };
        window.__setBBParams('time1');
        // 暂时去掉 umid 加载
        // window._APLUS_UMID_CLOSE_ = true;
    </script>
<link as="image" href="//s.alicdn.com/@sc04/kf/HLB15.FWU7voK1RjSZFNq6AxMVXaV.jpg_300x300.jpg" rel="preload"/>
<script>
            window.__BB_heroImgUrl = "//s.alicdn.com/@sc04/kf/HLB15.FWU7voK1RjSZFNq6AxMVXaV.jpg_300x300.jpg"
        </script>
<meta content="300" name="aplus-auto-exp-duration"/>
<!-- 暂时去掉加载 安全脚本 -->
<!--<meta name="aplus-xplug" cont